# Exploratory Data Analysis

This notebook explores the financial sentiment datasets and performs initial data analysis.


In [ ]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dataset_loader import load_dataset
from preprocess import clean_text, preprocess_batch

%matplotlib inline
plt.style.use('seaborn-v0_8')


## Load Dataset

Replace `data_path` with the actual path to your dataset file.


In [ ]:
# Load dataset
# data_path = 'data/financial_phrasebank.csv'  # Update with your path
# dataset_name = 'phrasebank'  # or 'semeval'

# df = load_dataset(dataset_name, data_path)
# print(f"Loaded {len(df)} samples")
# df.head()


## Label Distribution


In [ ]:
# Label distribution
# label_counts = df['label'].value_counts()
# print(label_counts)
# 
# plt.figure(figsize=(10, 6))
# label_counts.plot(kind='bar')
# plt.title('Label Distribution')
# plt.xlabel('Label')
# plt.ylabel('Count')
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()


## Text Length Analysis


In [ ]:
# Analyze text lengths
# df['text_length'] = df['text'].str.len()
# df['cleaned_length'] = df['cleaned_text'].str.len()
# 
# fig, axes = plt.subplots(1, 2, figsize=(14, 5))
# 
# axes[0].hist(df['text_length'], bins=50, edgecolor='black')
# axes[0].set_title('Original Text Length Distribution')
# axes[0].set_xlabel('Length (characters)')
# axes[0].set_ylabel('Frequency')
# 
# axes[1].hist(df['cleaned_length'], bins=50, edgecolor='black')
# axes[1].set_title('Cleaned Text Length Distribution')
# axes[1].set_xlabel('Length (characters)')
# axes[1].set_ylabel('Frequency')
# 
# plt.tight_layout()
# plt.show()
# 
# print(f"Mean original length: {df['text_length'].mean():.2f}")
# print(f"Mean cleaned length: {df['cleaned_length'].mean():.2f}")


## Sample Examples by Label


In [ ]:
# Display sample examples for each label
# for label in df['label'].unique():
#     print(f"\n{'='*60}")
#     print(f"Label: {label}")
#     print(f"{'='*60}")
#     samples = df[df['label'] == label]['text'].head(5)
#     for i, text in enumerate(samples, 1):
#         print(f"{i}. {text}")
